In [1]:
from selenium_crawl import SeleniumParser
import pickle
import time
import pandas as pd
from random import randint

In [2]:
sp = SeleniumParser(visible=True)

In [3]:
sp.browser.get('https://builtwith.com/login')
username = sp.browser.find_element_by_id("email")
password = sp.browser.find_element_by_id("password")
username.send_keys("ashwin@contractiq.com")
password.send_keys("ashwin0302")
sp.browser.find_element_by_name("ctl00$ctl00$content$contentAll$btnLogin").click()

In [4]:
# get trends page
soup = sp.get_soup('https://trends.builtwith.com/')

In [5]:
lis = soup.find('div',{'class':'titleBox'}).find('ul',{'class':'nav nav-zeit'}).find_all('li')[1:]

In [53]:
lis[0]

<li><a href="/ads" style="float:left;width:195px">Advertising</a>
<i class="cursor-pointer icon-chevron-down pull-right" onclick="if($(this).hasClass('icon-chevron-down')){$('._ads').fadeIn('fast');$(this).removeClass('icon-chevron-down'); $(this).addClass('icon-chevron-up');} else {$('._ads').fadeOut('fast'); $(this).removeClass('icon-chevron-up'); $(this).addClass('icon-chevron-down');}return false;" style="opacity:0.5;margin-top:3px"></i><div style="clear:both"></div></li>

In [54]:
lis[1]

<li class="_ads" style="display:none;background-color:#efefef"><a href="/ads/abandonment-ads" style="padding-left:20px">Abandonment Ads</a></li>

In [57]:
lis[0].get('class'),lis[1].get('class')

(None, ['_ads'])

In [22]:
lis[0].text,lis[0].find('a')['href'],lis[1].text,lis[1].find('a')['href']

(u'Advertising\n', '/ads', u'Abandonment Ads', '/ads/abandonment-ads')

In [6]:
tech_class_urls = [(li.text.strip(),'https://trends.builtwith.com'+li.find('a')['href'],
                        True if not li.get('class') else False) 
                   for li in lis]

In [7]:
tech_class_urls[:2]

[(u'Advertising', 'https://trends.builtwith.com/ads', True),
 (u'Abandonment Ads',
  'https://trends.builtwith.com/ads/abandonment-ads',
  False)]

In [48]:
soup1 = sp.get_soup(tech_class_urls[1][1])

In [51]:
soup1.find('div',{'class':'span8'}).find('tbody').findAll('tr')[-1].find('a').text

u'Realtime Targeting'

In [52]:
soup1.find('div',{'class':'span8'}).find('tbody').findAll('tr')[-1].find('a')['href']

'//trends.builtwith.com/ads/Realtime-Targeting'

In [49]:
soup1.find('div',{'class':'span8'}).find('tbody').findAll('tr')[-1].find_all('td')[1:][0].find('img').get('data-original-title')

'Increased 1 sites this week'

In [50]:
soup1.find('div',{'class':'span8'}).find('tbody').findAll('tr')[-1].find_all('td')[1:][0].get('value')

'1'

In [ ]:
final_list = []

In [21]:
for tech_sub_class,url,main_class_flag in tech_class_urls[164:]:
    if main_class_flag:
        tech_main_class = tech_sub_class
    print(tech_main_class,tech_sub_class,url)
    soup1 = sp.get_soup(url)
    try:
        tech_dets_trs = soup1.find('div',{'class':'span8'}).find('tbody').findAll('tr')
    except:
        print('error happened, no stats')
    out_list = []
    for tech_det_tr in tech_dets_trs:
        try:
            det_list = [tech_main_class,tech_sub_class]
            tech_name = tech_det_tr.find('a').text
            tech_url = tech_det_tr.find('a')['href']
            det_list.extend([tech_name,tech_url])
            tech_det_tds = tech_det_tr.find_all('td')[1:]
            for td in tech_det_tds:
                try:
                    det_list.append(td.get('value'))
                    det_list.append(td.find('img').get('data-original-title'))
                except:
                    det_list.append('')
                    det_list.append('')
            out_list.append(det_list)
        except:
            continue
    if out_list:
        final_list.extend(out_list)
    else:
        print('some error happened')
    with open('bw_tech_class_crawling_16mar.pkl','w') as f:
        pickle.dump(final_list,f)
    time.sleep(randint(3,7))

(u'Payment', u'Payment', 'https://trends.builtwith.com/payment')
(u'Payment', u'Acceptance Seal', 'https://trends.builtwith.com/payment/acceptance-seal')
(u'Payment', u'Alternative Checkout', 'https://trends.builtwith.com/payment/alternative-checkout')
(u'Payment', u'Bitcoin', 'https://trends.builtwith.com/payment/bitcoin')
(u'Payment', u'Checkout Buttons', 'https://trends.builtwith.com/payment/checkout-buttons')
(u'Payment', u'Fraud Prevention', 'https://trends.builtwith.com/payment/fraud-prevention')
(u'Payment', u'Micropayments', 'https://trends.builtwith.com/payment/micropayments')
(u'Payment', u'Mobile', 'https://trends.builtwith.com/payment/mobile')
(u'Payment', u'Payment Acceptance', 'https://trends.builtwith.com/payment/payment-acceptance')
(u'Payment', u'Payment Currency', 'https://trends.builtwith.com/payment/payment-currency')
(u'Payment', u'Payments Processor', 'https://trends.builtwith.com/payment/payments-processor')
(u'SEO Header Tag', u'SEO Header Tag', 'https://trends.

In [22]:
out_df = pd.DataFrame(final_list,columns=['tech_class','tech_sub_class','tech_name','tech_bw_url',
                                          '10k_sites','10k_change','100k_sites','100k_change',
                                         'million_sites','million_change','entire_sites','entire_change'])
out_df.shape

(8140, 12)

In [23]:
out_df.head()

,tech_class,tech_sub_class,tech_name,tech_bw_url,10k_sites,10k_change,100k_sites,100k_change,million_sites,million_change,entire_sites,entire_change
0,Advertising,Advertising,DoubleClick.Net,//trends.builtwith.com/ads/DoubleClick.Net,5732,"Increased 3,073 sites this week",45634,"Increased 30,336 sites this week",372577,"Increased 96,789 sites this week",1277851,"Increased 20,062 sites this quarter"
1,Advertising,Advertising,AppNexus,//trends.builtwith.com/ads/AppNexus,3608,"Increased 1,349 sites this week",21340,"Increased 10,850 sites this week",78746,"Increased 25,943 sites this week",24433,Increased 99 sites this quarter
2,Advertising,Advertising,Google Adsense,//trends.builtwith.com/ads/Google-Adsense,3073,Increased 781 sites this week,23489,"Increased 6,509 sites this week",174892,"Increased 11,304 sites this week",14162806,"Increased 108,099 sites this quarter"
3,Advertising,Advertising,Rubicon Project,//trends.builtwith.com/ads/Rubicon-Project,2950,Increased 797 sites this week,16105,"Increased 4,720 sites this week",52368,"Increased 8,898 sites this week",6583,Increased 541 sites this quarter
4,Advertising,Advertising,Facebook Custom Audiences,//trends.builtwith.com/ads/Facebook-Custom-Aud...,2788,Increased 745 sites this week,16261,"Increased 4,558 sites this week",95112,"Increased 11,325 sites this week",814574,"Increased 97,169 sites this quarter"


In [24]:
out_df.to_csv('bw_tech_classification_march_2017.csv',index=False,quoting=1,encoding='utf-8')

In [25]:
sp.exit()